In [1]:
import pandas as pd
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import random
from operator import itemgetter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [2]:
# Load Train data 
df_train_prepared = pd.read_pickle('df_train_prepared')
 

train_payprice = df_train_prepared['payprice']
train_bidprice = df_train_prepared['bidprice']
train_click = df_train_prepared['click']

rtb_train = df_train_prepared.drop(['click','payprice','bidprice'], axis =1)
rtb_train.head()



,weekday,hour,region,city,adexchange,slotwidth,slotheight,slotprice,advertiser,useragent_encoded,IP_encoded,url_encoded,domain_encoded,slotid_encoded,slotvisibility_encoded,slotformat_encoded,creative_encoded,keypage_encoded,usertag_encoded
0,5,22,2,2,2,200,200,5,3427,31,253587,268016,21783,12845,2,0,104,0,744035
1,1,20,238,239,1,300,250,0,2821,29,282720,145905,728,50771,6,3,9,18,744035
2,3,13,40,41,2,250,250,5,3427,31,469429,262017,18426,9136,2,0,92,0,188248
3,6,23,1,1,1,160,600,0,1458,31,107833,658015,4770,50450,2,1,111,12,706361
4,5,6,216,233,2,728,90,133,2259,31,342191,157670,153,1517,8,3,84,18,744035


In [10]:
# Scale validation data for better prediction when using logistic 
standard_scaler = StandardScaler().fit(rtb_train)
train_x = standard_scaler.transform(rtb_train)


In [11]:
# Load validation data
rtb_test = pd.read_pickle('df_test_prepared')

#val_payprice = df_val_prepared['payprice']
#val_bidprice = df_val_prepared['bidprice']
#val_click = df_val_prepared['click']

#rtb_validation = df_val_prepared.drop(['click','payprice','bidprice'], axis =1)
rtb_test.head()

,weekday,hour,region,city,adexchange,slotwidth,slotheight,slotprice,advertiser,useragent_encoded,IP_encoded,url_encoded,domain_encoded,slotid_encoded,slotvisibility_encoded,slotformat_encoded,creative_encoded,keypage_encoded,usertag_encoded
0,0,12,146,159,3,300,250,10,3427,24,167628,18581,3258,13944,0,0,1,11,17374
1,3,14,1,1,0,320,50,5,2997,5,92907,70008,4693,6255,5,3,13,18,115970
2,5,19,27,34,1,336,280,0,1458,26,48596,57287,6951,15499,2,1,89,12,107970
3,0,21,238,245,4,960,90,30,2821,26,46570,27487,2378,3922,5,3,4,18,72990
4,2,20,27,35,3,300,250,50,3386,26,150125,23871,819,14006,0,0,50,3,53930


In [12]:
# Scale validation data for better prediction when using logistic 
standard_scaler = StandardScaler().fit(rtb_test)
test_x = standard_scaler.transform(rtb_test)


### Average CTR for Train

### RANDOM FOREST

Random Forest produce slightly better prediction

In [20]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(train_x_rus, train_y_rus)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### PREDICTION TRAIN WITH RANDOM FOREST

In [21]:
# Prediction for train
train_tree_pred = tree_clf.predict(train_x)
train_tree_prob = tree_clf.predict_proba(train_x)

tree_error = sum(train_tree_pred != train_click) 
tree_correct = sum(train_tree_pred == train_click) 
print('Wrong Predictions from Decision Tree For Validation:', tree_error)


Wrong Predictions from Decision Tree For Validation: 589299


In [27]:
# Put in df
train_tree_predicted = pd.DataFrame(columns=['probability_no_click','probability_click'], data = train_tree_prob)
train_tree_predicted['click'] = train_tree_pred
train_tree_predicted[840:845]

,probability_no_click,probability_click,click
840,0.613445,0.386555,0
841,0.613445,0.386555,0
842,0.613445,0.386555,0
843,0.434842,0.565158,1
844,0.613445,0.386555,0


### PREDICTION TEST DATA WITH RANDOM FOREST

In [28]:
# Prediction for validation
test_prediction = tree_clf.predict(test_x)
test_prob = tree_clf.predict_proba(test_x)


In [29]:
# Put in df
df_test_prediction = pd.DataFrame(columns=['probability_no_click','probability_click'], data = test_prob)

df_test_prediction['click'] = test_prediction
df_test_prediction.to_pickle('df_test_prediction')
df_test_prediction[840:845]

,probability_no_click,probability_click,click
840,0.613445,0.386555,0
841,0.613445,0.386555,0
842,0.613445,0.386555,0
843,0.613445,0.386555,0
844,0.434842,0.565158,1


In [69]:

pCTR_test = df_test_prediction['probability_click']
df_test_prediction.to_pickle('pCTR_train')


### Linear Bidding 

basebid * pCRT/averageCTR 

pCTR = probability of click from Decision Tree Model 

In [103]:
click = df_train_prepared[df_train_prepared['click'] == 1]
avgCTR = len(click)/len(df_train_prepared)

AvgBidPrice = train_bidprice.mean()
AvgBidPrice_oneimp  = AvgBidPrice/1000

std = np.std(train_bidprice)/1000
std

0.02891152703068351

In [87]:
base= AvgBidPrice/1000 + std
base = float("{0:.2f}".format(base))
base 

0.3

In [113]:
# Fede Bid strategy 
# Base bid = meidan 

avgCTR = len(click)/len(df_train_prepared)
base_bid =  84/1000
pCTR = pCTR_test
bidPrice = base_bid * (pCTR/avgCTR)
bidPrice

0          44.024196
1         102.423377
2          44.024196
3          44.024196
4          44.024196
5          44.024196
6          64.365070
7          44.024196
8          64.365070
9          44.024196
10         44.024196
11        102.423377
12         44.024196
13         64.365070
14         44.024196
15         44.024196
16         44.024196
17         64.365070
18         44.024196
19         44.024196
20         64.365070
21         44.024196
22         44.024196
23         64.365070
24         44.024196
25        102.423377
26         44.024196
27         44.024196
28         44.024196
29         44.024196
             ...    
303345     44.024196
303346     44.024196
303347     44.024196
303348     44.024196
303349     64.365070
303350     44.024196
303351     64.365070
303352     64.365070
303353     66.666545
303354     44.024196
303355     44.024196
303356     64.365070
303357     44.024196
303358     44.024196
303359     44.024196
303360     44.024196
303361     44

In [114]:
 

test_prediction_bid = base * (pCTR_test/AvgCTR)
#test_prediction_bid = test_prediction_bid.to_frame()
test_prediction_bid.head()

0    157.229271
1    365.797775
2    157.229271
3    157.229271
4    157.229271
Name: probability_click, dtype: float64

###  Take the bidid and combining with the bid price prediction

In [99]:
# Take the bidid and combining with the bid price prediction 
bidid = pd.read_pickle('df_test_bidid')
bidid.head()

0    366c563de7d90feb9d4dab53e795a93fb3157387
1    29167d4caa719788b5a342dbaa25151d53121f80
2    ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e
3    844c2da00d45315f20b748ec131c26ee99a7cbc7
4    c6017f0ad0c44d7d0c9b62583ea863f28941c0ca
Name: bidid, dtype: object

In [119]:
linear_prediction =  pd.concat([bidid, bidPrice], axis=1, join='inner')
linear_prediction.columns = ['bidid', 'linear_bid']
linear_prediction.head()


,bidid,linear_bid
0,366c563de7d90feb9d4dab53e795a93fb3157387,44.024196
1,29167d4caa719788b5a342dbaa25151d53121f80,102.423377
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,44.024196
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,44.024196
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,44.024196


In [120]:
linear_prediction.columns = ['bidid', 'linear_bid']
linear_prediction.head()

,bidid,linear_bid
0,366c563de7d90feb9d4dab53e795a93fb3157387,44.024196
1,29167d4caa719788b5a342dbaa25151d53121f80,102.423377
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,44.024196
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,44.024196
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,44.024196


In [121]:
linear_prediction.to_csv('Group10_linear_prediction_trial2.csv')